In [0]:
%sql
USE CATALOG catalogo_energia;

In [0]:
%sql
-- Checagem de Volume: contagem de linhas por tabela Bronze
SELECT 'consumo_geral' AS tabela, COUNT(*) AS total_linhas FROM catalogo_energia.bronze.consumo_geral
UNION ALL
SELECT 'energia_limpa' AS tabela, COUNT(*) AS total_linhas FROM catalogo_energia.bronze.energia_limpa;

In [0]:
# Ajuste a lista de colunas-chave conforme seu domínio (ex.: id_linha, data_leitura, id_medição, etc.)
possible_keys = {
    "catalogo_energia.bronze.consumo_geral": ["id_leitura", "data_registro", "medidor_id"],
    "catalogo_energia.bronze.energia_limpa": ["id", "data", "site_id"]
}

dup_report = []
for full_name, keys in possible_keys.items():
    # filtra apenas chaves que existem na tabela
    df = spark.table(full_name)
    existing_keys = [k for k in keys if k in df.columns]
    if not existing_keys:
        dup_report.append({"tabela": full_name, "status":"no_key_columns_found", "keys_tried": keys})
        continue
    grp = df.groupBy(existing_keys).count().filter("count > 1")
    n_dup_keys = grp.count()
    dup_report.append({
        "tabela": full_name,
        "keys_used": existing_keys,
        "duplicated_combinations": n_dup_keys
    })

import json
print(json.dumps(dup_report, indent=2))


In [0]:
%sql
-- Amostras com higienização técnica (visual) - Consumo Geral
SELECT * FROM catalogo_energia.bronze.consumo_geral LIMIT 5;

In [0]:

%sql
-- Amostras com higienização técnica (visual) - Energia Limpa
SELECT * FROM catalogo_energia.bronze.energia_limpa LIMIT 5;

In [0]:
df = spark.table("catalogo_energia.bronze.consumo_geral")
df.printSchema()

df2 = spark.table("catalogo_energia.bronze.energia_limpa")
df2.printSchema()

In [0]:
from pyspark.sql import functions as F

for full_table in [
    "catalogo_energia.bronze.consumo_geral", 
    "catalogo_energia.bronze.energia_limpa"
]:
    df = spark.table(full_table)

    # verificar todas as variações possíveis
    for col in ["Regiao", "Região", "region", "Region", "regiao"]:
        if col in df.columns:
            print(f"\nTabela: {full_table} — coluna encontrada: {col}")
            df.groupBy(col).count().orderBy(F.desc("count")).show(50, truncate=False)
            break
    else:
        print(f"\nTabela: {full_table} -> nenhuma coluna de região encontrada.")


In [0]:
from pyspark.sql import functions as F
import pandas as pd

# -----------------------------
# 1-  Listar as tabelas Bronze
# -----------------------------
tables = [
    ("consumo_geral", "catalogo_energia.bronze.consumo_geral"),
    ("energia_limpa", "catalogo_energia.bronze.energia_limpa")
]

report_rows = []

for short_name, full_name in tables:
    df = spark.table(full_name)

    # contagem total
    total = df.count()

    # total de colunas
    num_cols = len(df.columns)

    # amostra de primeiras colunas
    preview = df.limit(1).toPandas().to_dict(orient="records")[0]

    # montar linha do relatório
    report_rows.append({
        "tabela": full_name,
        "total_linhas": total,
        "qtd_colunas": num_cols,
        "colunas": ", ".join(df.columns),
        "amostra_primeira_linha": str(preview),
    })

# converter para pandas → spark
pdf = pd.DataFrame(report_rows)
report_df = spark.createDataFrame(pdf)

# adicionar timestamp da execução
report_df = report_df.withColumn("executado_em", F.current_timestamp())


# -----------------------------------------
# 2) Salvar como tabela Delta da Bronze QA
# -----------------------------------------
report_df.write.format("delta").mode("overwrite").saveAsTable(
    "catalogo_energia.bronze.validacao_bronze_report"
)

print("✅ Relatório salvo com sucesso em:")
print("catalogo_energia.bronze.validacao_bronze_report")

display(report_df)

In [0]:
%sql
USE CATALOG catalogo_energia;

SELECT *
FROM catalogo_energia.bronze.validacao_bronze_report
ORDER BY executado_em DESC;